## Prueba base

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

model_path = "clasificador_analisis/clasificador/clasificador_tono/class_balanced/comparativa/ep_4/modelo_final"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


In [ ]:
def predict_sentiment_with_neutral(text, model, tokenizer, alpha=1.3, neutral_threshold=0.6):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).squeeze().tolist()

    neutral_score = 1 - abs(probs[0] - 0.5) ** alpha - abs(probs[2] - 0.5) ** alpha

    if neutral_score > (1 - neutral_threshold):
        pred_label = 1  
    else:
        pred_label = int(torch.argmax(torch.tensor(probs)))

    return pred_label, probs


### Grupal

In [ ]:
df = pd.read_excel("clasificador_analisis/clasificador/clasificador_tono/class_balanced/mejor_test.xlsx")
df = df.rename(columns={"text": "text", "label": "label_manual_num"})

In [8]:

preds_and_scores = df["text"].apply(lambda x: predict_sentiment_with_neutral(x, model, tokenizer))
df["label_predicted"] = preds_and_scores.apply(lambda x: x[0])
df["scores"] = preds_and_scores.apply(lambda x: x[1])
df[["score_negativo", "score_neutro", "score_positivo"]] = pd.DataFrame(df["scores"].tolist(), index=df.index)

# Métricas
print("\n📊 MATRIZ DE CONFUSIÓN:")
print(confusion_matrix(df["label_manual_num"], df["label_predicted"]))

print("\n📈 CLASIFICATION REPORT:")
print(classification_report(
    df["label_manual_num"],
    df["label_predicted"],
    target_names=["Negativo", "Neutro", "Positivo"],
    digits=3
))



📊 MATRIZ DE CONFUSIÓN:
[[79 18  4]
 [10 83  8]
 [ 1 11 89]]

📈 CLASIFICATION REPORT:
              precision    recall  f1-score   support

    Negativo      0.878     0.782     0.827       101
      Neutro      0.741     0.822     0.779       101
    Positivo      0.881     0.881     0.881       101

    accuracy                          0.828       303
   macro avg      0.833     0.828     0.829       303
weighted avg      0.833     0.828     0.829       303

